# AI 生成图片

本手册演示了如何使用 `OpenAI DALL-E 2` 生成图像，并结合其他LLM功能，如文本和嵌入生成。

这里，我们使用`Text Generation` 生成随机图像描述，并使用 `dall - e2` 从该描述创建图像，以内联方式显示图像。

最后，笔记本要求用户描述图像。使用余弦相似度将用户描述的嵌入与原始描述进行比较，并返回从0到1的分数，其中1表示完全匹配。

In [ ]:
// Usual setup: importing Semantic Kernel SDK and SkiaSharp, used to display images inline.

#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"
#r "nuget: SkiaSharp, 2.88.3"

#!import config/Settings.cs
#!import config/Utils.cs
#!import config/SkiaUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.AI.ImageGeneration; 
using Microsoft.SemanticKernel.AI.Embeddings;
using Microsoft.SemanticKernel.AI.Embeddings.VectorOperations;

## 设置，使用 AI 服务：图像，文本，嵌入资源

该笔记使用了：

- text-davinci-003 生成随机图片描述
- OpenAI Dall-E 2 将图片描述转换为图片
- text-embedding-data-002 将你的猜测与真实图像描述进行比较

In [ ]:
// Load OpenAI credentials from config/settings.json
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

// Configure the three AI features: text embedding (using Ada), text completion (using DaVinci 3), image generation (DALL-E 2)
var kernel = new KernelBuilder()
    .Configure(c =>
    {
        c.AddOpenAITextEmbeddingGenerationService("text-embedding-ada-002", apiKey);
        c.AddOpenAITextCompletionService("text-davinci-003", apiKey, orgId);
        c.AddOpenAIImageGenerationService(apiKey, orgId);
    })
    .Build();

// Get AI service instance used to generate images
var dallE = kernel.GetService<IImageGeneration>();

// Get AI service instance used to extract embedding from a text
var textEmbedding = kernel.GetService<IEmbeddingGeneration<string, float>>();

## 通过 DALL-E 2 生成图片

`genImgDescription ` 是使用一个语义函数来生成一个随机图片描述。该函数接受一个随机数作为输入，以增加输出的多样性。

然后将随机图像描述提供给 `dall - e 2`，要求其创建图像。

In [ ]:
// Create a semantic function that generate a random image description.
var genImgDescription = kernel.CreateSemanticFunction(
    "Think about an artificial object correlated to number {{$input}}. " +
    "Describe the image with one detailed sentence. The description cannot contain numbers.", 
    maxTokens: 256, temperature: 1);

var random = new Random().Next(0, 200);
var imageDescription = await genImgDescription.InvokeAsync($"{random}");

// Use DALL-E 2 to generate an image. OpenAI in this case returns a URL (though you can ask to return a base64 image)
var imageUrl = await dallE.GenerateImageAsync(imageDescription.Result.Trim(), 512, 512);

await SkiaUtils.ShowImage(imageUrl, 512, 512);

## 猜谜游戏

让我们来看描述猜画游戏吧。

你最后会得到一个分数。

In [ ]:
// Prompt the user to guess what the image is
var guess = await InteractiveKernel.GetInputAsync("Describe the image in your words");

// Compare user guess with real description and calculate score
var origEmbedding = await textEmbedding.GenerateEmbeddingsAsync(new List<string> { imageDescription.Result} );
var guessEmbedding = await textEmbedding.GenerateEmbeddingsAsync(new List<string> { guess } );
var similarity = origEmbedding.First().AsReadOnlySpan().CosineSimilarity(guessEmbedding.First().AsReadOnlySpan());

Console.WriteLine($"Your description:\n{Utils.WordWrap(guess, 90)}\n");
Console.WriteLine($"Real description:\n{Utils.WordWrap(imageDescription.Result.Trim(), 90)}\n");
Console.WriteLine($"Score: {similarity:0.00}\n\n");

//Uncomment this line to see the URL provided by OpenAI
//Console.WriteLine(imageUrl);

In [ ]:
// 结果例子

Your description:
a blue and yellow toy ca over a squared blue and yellow surface


Real description:
A bright yellow plastic toy car with a blue stripe down the side and a white star on the
hood, sitting atop a checkered patterned rug.


Score: 0.92